In [17]:
import sys; sys.path.append("../")

In [18]:
from dataset_wrappper import NewsData
from utils_train import transfer_batch_to_device, load_from_checkpoint
import os
from pathlib import Path
from train import get_model_on_device
from transformers import RobertaTokenizerFast
import torch
from utils_evaluation import tokenizer_batch_decode
import torch.nn.functional as F
from utils_evaluation import tokenizer_batch_decode
from run_validation import load_model_for_eval
import copy
import pickle

%matplotlib inline

In [19]:
BATCH_SIZE = 64
DEVICE = "cuda:0"
CHECKPOINT_TYPE = "best" # else "best"
RESULT_DIR = Path("result-files")

In [20]:
data = NewsData(batch_size=BATCH_SIZE, tokenizer_name="roberta", dataset_name="ptb_text_only", max_seq_len=64)
validation_loader = data.val_dataloader(batch_size=BATCH_SIZE)
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

print(f"{len(validation_loader)} batches with batch size {BATCH_SIZE}")

Is file!
train 42068
validation 3370
test 3761
53 batches with batch size 64


In [21]:
def get_clean_name(run_name):
    latent_size = run_name.split("-")[4][-2:]
    if "autoencoder" in run_name:
        FB = "autoencoder"
    else:
        FB = run_name.split("-")[6]
        if len(FB) == 3:
            FB += "0"
        FB = "FB-" + FB
    clean_name = f"NZ-{latent_size} | {FB}"
    return clean_name

In [22]:
PTB_run_name_paths = {}
for r in os.listdir("../Runs"):
    if "PTB" in r:
        path = Path("../Runs") / r / f"checkpoint-{CHECKPOINT_TYPE}.pth"
        PTB_run_name_paths[r] = path

clean_names = []
for r in PTB_run_name_paths.keys():
#     print(get_clean_name(r))
    clean_names.append(get_clean_name(r))
    
clean_names = sorted(clean_names)

In [28]:

results_exact_match = {}

for AUTO_REGRESSIVE in [True, False]:
    MODE = 'TF' if AUTO_REGRESSIVE is False else 'AR'
    RESULT_FILE = RESULT_DIR / f"result_exact_match_{MODE}.pickle"

    if os.path.exists(RESULT_FILE):
        print(f"Loading file {RESULT_FILE}, it existed.")
        res_dict = pickle.load( open( RESULT_FILE, "rb" ) )
    else:
        res_dict = {}
    
    for i, (r, p) in enumerate(PTB_run_name_paths.items()):
        print(i, r)
        
        if r in res_dict:
            print("Skipping this run, already in result dict!")
            continue

        exact_match_acc = []
        exact_match_over_seq = []
        label_masks = []

        latent_size = int(r.split("-")[4][-2:])

        vae_model = load_model_for_eval(p, device_name="cuda:0", 
                                        latent_size=latent_size, 
                                        add_latent_via_memory=True,
                                        add_latent_via_embeddings=False, 
                                        do_tie_weights=True, 
                                        do_tie_embedding_spaces=True,
                                        add_decoder_output_embedding_bias=False)


        N = len(validation_loader)
        with torch.no_grad():
            for batch_i, batch in enumerate(validation_loader):
                print(f"{batch_i:3d}/{N}", end="\r")
                batch = transfer_batch_to_device(batch, DEVICE)
                
                labels = copy.deepcopy(batch["input_ids"].cpu())
                label_mask = labels[:, 1:].contiguous()
                # pad token is int 1
                label_mask = (label_mask != 1).float()
                
                vae_output = vae_model.forward(batch["input_ids"], 
                                               1.0, 
                                               batch["attention_mask"],
                                               auto_regressive=AUTO_REGRESSIVE,
                                               objective="beta-vae",
                                               hinge_kl_loss_lambda=0.0,
                                               return_cross_entropy=True,
                                               return_exact_match=True,
                                               reduce_seq_dim_exact_match="none",
                                               reduce_batch_dim_exact_match="none",
                                               nucleus_sampling=False,
                                               device_name="cuda:0",
                                               decode_sample_from_prior=False)

                exact_match_over_seq.append(vae_output["exact_match"].cpu())
                label_masks.append(label_mask)

                acc = vae_output["exact_match"].cpu() * label_mask
                acc = acc.sum(dim=1) / label_mask.sum(dim=1)
                exact_match_acc.append(acc)
        
        max_len = max([t.shape[1] for t in exact_match_over_seq])
        
        exact_match_over_seq = [torch.nn.functional.pad(a, (0, max_len - a.shape[1]), mode='constant', value=0) for a in exact_match_over_seq]
        exact_match_over_seq = torch.cat(exact_match_over_seq, dim=0)
        
        label_masks = [torch.nn.functional.pad(a, (0, max_len - a.shape[1]), mode='constant', value=0) for a in label_masks]
        label_masks = torch.cat(label_masks, dim=0)
       
        exact_match_acc = torch.cat(exact_match_acc, dim=0)
        
        res_dict[r] ={
            "exact_match_acc": exact_match_acc.numpy(),
            "exact_match_over_seq": exact_match_over_seq.numpy(),
            "label_masks": label_masks.numpy()
        }

        pickle.dump( res_dict, open( RESULT_FILE, "wb" ) )
    
    results_exact_match[MODE] = res_dict

Loading file result-files/result_exact_match_AR.pickle, it existed.
0 2021-02-03-PTB-latent32-FB-0.5-run-09:31:02
Skipping this run, already in result dict!
1 2021-02-03-PTB-latent32-FB-1.0-run-11:43:17
Skipping this run, already in result dict!
2 2021-02-03-PTB-latent32-FB-1.50-run-12:13:36
Skipping this run, already in result dict!
3 2021-02-03-PTB-latent64-FB-1.0-run-13:06:00
Skipping this run, already in result dict!
4 2021-02-03-PTB-latent32-FB-0.00-run-14:32:09
Skipping this run, already in result dict!
5 2021-02-03-PTB-latent32-autoencoder-run-17:30:41
Skipping this run, already in result dict!
6 2021-02-03-PTB-latent64-FB-0.50-run-12:29:58
Skipping this run, already in result dict!
7 2021-02-03-PTB-latent64-FB-1.50-run-13:22:14
Skipping this run, already in result dict!
8 2021-02-03-PTB-latent64-FB-0.00-run-17:14:10
Skipping this run, already in result dict!
9 2021-02-03-PTB-latent64-autoencoder-run-18:25:57
Skipping this run, already in result dict!
10 2021-02-02-PTB-latent32-

Some weights of the model checkpoint at roberta-base were not used when initializing VAE_Encoder_RobertaModel: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing VAE_Encoder_RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VAE_Encoder_RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VAE_Encoder_RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tying encoder decoder RoBERTa checkpoint weights!
<class 'modules.decoder_roberta_new.VaeDecoderRobertaModel'> and <class 'modules.encoder_roberta.VAE_Encoder_RobertaModel'> are not equal. In this case make sure that all encoder weights are correctly initialized. 
The following encoder weights were not tied to the decoder ['roberta/pooler']
Tying embedding spaces!
Done model...
Loading VAE_model, optimizer and scheduler from ../Runs/2021-02-03-PTB-latent32-FB-0.5-run-09:31:02/checkpoint-best.pth
Removing module string from state dict from checkpoint
Checkpoint global_step: best, epoch: 37, best_valid_loss: 84.10610651086878


/home/cbarkhof/.conda/envs/thesisenv/lib/python3.6/site-packages/datasets/arrow_dataset.py:847: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)
/home/cbarkhof/.conda/envs/thesisenv/lib/python3.6/site-packages/datasets/arrow_dataset.py:847: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor.

1 2021-02-03-PTB-latent32-FB-1.0-run-11:43:17
Loading model...
Replacing linear output layer with one without bias!


...

/home/cbarkhof/.conda/envs/thesisenv/lib/python3.6/site-packages/datasets/arrow_dataset.py:847: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)
/home/cbarkhof/.conda/envs/thesisenv/lib/python3.6/site-packages/datasets/arrow_dataset.py:847: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor.

13 2021-02-02-PTB-latent64-FB-0.25-run-13:17:02
Loading model...
Replacing linear output layer with one without bias!


Some weights of the model checkpoint at roberta-base were not used when initializing VAE_Encoder_RobertaModel: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing VAE_Encoder_RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VAE_Encoder_RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VAE_Encoder_RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tying encoder decoder RoBERTa checkpoint weights!
<class 'modules.decoder_roberta_new.VaeDecoderRobertaModel'> and <class 'modules.encoder_roberta.VAE_Encoder_RobertaModel'> are not equal. In this case make sure that all encoder weights are correctly initialized. 
The following encoder weights were not tied to the decoder ['roberta/pooler']
Tying embedding spaces!
Done model...
Loading VAE_model, optimizer and scheduler from ../Runs/2021-02-02-PTB-latent64-FB-0.25-run-13:17:02/checkpoint-best.pth
Removing module string from state dict from checkpoint
Checkpoint global_step: best, epoch: 37, best_valid_loss: 84.84688652886285


/home/cbarkhof/.conda/envs/thesisenv/lib/python3.6/site-packages/datasets/arrow_dataset.py:847: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)
/home/cbarkhof/.conda/envs/thesisenv/lib/python3.6/site-packages/datasets/arrow_dataset.py:847: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor.